Funktionen aus US1

In [7]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Rekursive Dateisuche nach .tsv-Dateien
def find_tsv_files(directory):
    tsv_files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tsv'):
                full_path = os.path.join(root, file)
                tsv_files_list.append(full_path)
    return tsv_files_list

Um die Ergebnisse von User Story 2 darzustellen, wurde auf die Visualisierung aus User Story 1 verzichtet.

In [8]:
# Funktion zum markieren von Ausreißern
def mark_outliers(file_path, method='iqr', threshold=1.5):

    try:
        # Einlesen der TSV-Datei
        df = pd.read_csv(file_path, sep='\t')

        # Einlesen der Spaltennamen
        gene_col = df.columns[0] # erste Spalte ist immer das Gen
        symbol_col = df.columns[-1] # letzte Spalte ist immer das Symbol
        entity_col = df.columns[-2] # vorletzte Spalte ist immer die Entity

        # Count-Spalten extrahieren
        count_cols = df.columns[1:-2]

        # DataFrame für Ergebnisse initialisieren
        results_df = df.copy()
        results_df['is_outlier'] = False
        results_df['outlier_info']  = ""

        # Gruppierung nach Entity (host/phage)
        host_mask = df[entity_col] == 'host'
        phage_mask = df[entity_col] == 'phage'

        # Anzahl der Host- und Phagen-Gene
        #print(f"Identifizierte Gene: {sum(host_mask)} Host-Gene, {sum(phage_mask)} Phagen-Gene")

       # Ausreißer für Host-Gene identifizieren
        if sum(host_mask) > 0:
            host_outliers = detect_outliers(df[host_mask], count_cols, method, threshold)
            for idx in host_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Host-Outlier; "

        # Ausreißer für Phagen-Gene identifizieren
        if sum(phage_mask) > 0:
            phage_outliers = detect_outliers(df[phage_mask], count_cols, method, threshold)
            for idx in phage_outliers:
                results_df.loc[idx, 'is_outlier'] = True
                results_df.loc[idx, 'outlier_info'] += "Phagen-Outlier; "

        return results_df

    except Exception as e:
        print(f"Fehler {e}")
        return None

# Hilfsfunktion zur Ausreißererkennung
def detect_outliers(df, count_cols, method='iqr', threshold=1.5):
    outlier_indices = set()

    if method == 'iqr':
        # Interquartilsabstand-Methode
        for col in count_cols:
            Q05 = df[col].quantile(0.05)
            Q1 = df[col].quantile(0.25)
            #print(f"Q1: {Q1}")
            Q3 = df[col].quantile(0.75)
            #print(f"Q3: {Q3}")
            IQR = Q3 - Q1
            #print(f"IQR: {IQR}")
            lower_bound = Q1 - threshold * IQR
            #print(f"Lower Bound: {lower_bound}")
            upper_bound = Q3 + threshold * IQR
            #print(f"Upper Bound: {upper_bound}")
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.update(outliers)

         # Berechne das 0.05-Quantil für jede Spalte
        quantiles_05 = {col: df[col].quantile(0.05) for col in count_cols}

        # Prüfe für jede Zeile, ob alle Werte kleiner als das 0.05-Quantil sind
        for idx, row in df.iterrows():
            if all(row[col] < quantiles_05[col] for col in count_cols):
                outlier_indices.add(idx)

    elif method == 'zscore':
        # Z-Score-Methode
        for col in count_cols:
            z_scores = stats.zscore(df[col], nan_policy='omit')
            outliers = df[abs(z_scores) > threshold].index
            outlier_indices.update(outliers)

    return outlier_indices


### Datenbereinigung  

Als Nächstes definieren wir eine Funktion `clean_outlier_samples`, die potenzielle Ausreißer erkennt und entfernt. Diese Funktion basiert auf der vorherigen Methode zur Ausreißererkennung (`mark_outliers`), entfernt jedoch direkt im Anschluss die betroffenen Zeilen und die dazugehörigen Spalten `is_outlier` und `outlier_info`.

Die bereinigte Version jeder Datei wird unter demselben Namen mit dem Suffix `_cleaned.tsv` im Ordner `./cleaned_data/` gespeichert. Dadurch bleibt der ursprüngliche Datensatz unverändert erhalten und die finalen, bereinigten Daten sind sofort einsatzbereit.

Für die gleichzeitige Bereinigung mehrerer Dateien definieren wir zusätzlich eine Batch-Funktion `batch_clean_all_tsv`. Diese durchläuft alle `.tsv`-Dateien in einem angegebenen Verzeichnis, bereinigt sie nacheinander mit der beschriebenen Methode und speichert alle bereinigten Dateien im Zielordner.

In [9]:
import glob

#Funktion zur Datenbereinigung
def clean_outlier_samples(file_path, method='iqr', threshold=1.5, output_dir="../cleaned_data"):
    cleaned_df = mark_outliers(file_path, method=method, threshold=threshold)

    if cleaned_df is None:
        print(f"Fehler beim Verarbeiten der Datei: {file_path}")
        return

    # Count the number of outliers
    num_outliers = cleaned_df["is_outlier"].sum()

    # Remove outliers and drop related columns
    cleaned_df = cleaned_df[~cleaned_df["is_outlier"]].copy()
    cleaned_df.drop(columns=["is_outlier", "outlier_info"], inplace=True, errors='ignore')

    # Save the cleaned file
    os.makedirs(output_dir, exist_ok=True)
    file_name = os.path.basename(file_path).replace(".tsv", "_cleaned.tsv")
    save_path = os.path.join(output_dir, file_name)
    cleaned_df.to_csv(save_path, sep='\t', index=False)

    # Output result summary
    print(f"{num_outliers} Ausreißer wurden entfernt → {file_name}")
    print(f"Finalisierte Datei gespeichert: {save_path}")

    return cleaned_df


def batch_clean_all_tsv(input_dir, output_dir, method='iqr', threshold=1.5):
    tsv_files = find_tsv_files(input_dir)

    if not tsv_files:
        print(f"Keine TSV-Dateien gefunden in: {input_dir}")
        return

    print(f"{len(tsv_files)} Dateien werden verarbeitet...\n")
    for file_path in tsv_files:
        clean_outlier_samples(
            file_path=file_path,
            method=method,
            threshold=threshold,
            output_dir=output_dir
        )
    print("\nAlle Dateien wurden verarbeitet.")


Zum Abschluss der Datenbereinigung möchten wir einen schnellen Überblick über die bereinigten Datensätze erhalten. Dazu verwenden wir die Funktion `preview_tsv_files`, die eine definierte Anzahl an Zeilen aus allen `.tsv`-Dateien in einem angegebenen Verzeichnis anzeigt.  

Diese Vorschau hilft dabei, die Struktur der Dateien zu kontrollieren und sicherzustellen, dass die Spalten `is_outlier` und `outlier_info` (sofern entfernt) nicht mehr enthalten sind.  

Zunächst rufen wir `batch_clean_all_tsv()` auf, um alle `.tsv`-Dateien im Eingabeverzeichnis zu bereinigen. Anschließend verwenden wir `preview_tsv_files`, um die bereinigten Dateien (`_cleaned.tsv`) zu betrachten.  

In [10]:
from IPython.display import display

def preview_tsv_files(directory, suffix="_cleaned.tsv", preview_rows=3):
    files = [f for f in os.listdir(directory) if f.endswith(suffix)]

    if not files:
        print(f"Keine Dateien mit dem Suffix '{suffix}' gefunden in: {directory}")
        return

    for file in files:
        print(f"{file}")
        df = pd.read_csv(os.path.join(directory, file), sep='\t')
        display(df.head(preview_rows))

In [11]:
#directory address
input_dir = '../data/raw_data_bb'
output_dir = '../data/cleaned_data_bb'

batch_clean_all_tsv(input_dir, output_dir)
print()
print("Previewing files without outliers removed (without 'is_outlier' and 'outlier_info' columns)")
preview_tsv_files(output_dir, "_cleaned.tsv")

7 Dateien werden verarbeitet...

1863 Ausreißer wurden entfernt → Brandao_LB_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/cleaned_data_bb\Brandao_LB_full_raw_counts_cleaned.tsv
1439 Ausreißer wurden entfernt → Ceyssens_non-directional_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/cleaned_data_bb\Ceyssens_non-directional_full_raw_counts_cleaned.tsv
892 Ausreißer wurden entfernt → Guegler_T7_minusToxIN_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/cleaned_data_bb\Guegler_T7_minusToxIN_full_raw_counts_cleaned.tsv
805 Ausreißer wurden entfernt → Leskinen_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/cleaned_data_bb\Leskinen_full_raw_counts_cleaned.tsv
747 Ausreißer wurden entfernt → Li_full_raw_counts_cleaned.tsv
Finalisierte Datei gespeichert: ../data/cleaned_data_bb\Li_full_raw_counts_cleaned.tsv
684 Ausreißer wurden entfernt → Sprenger_VC_delta_tdh_VP882_WT_full_raw_counts_cleaned.tsv
Finalisierte D

,Geneid,0_R1,0_R2,0_R3,5_R1,5_R2,5_R3,10_R1,10_R2,10_R3,15_R1,15_R2,15_R3,Entity,Symbol
0,gene-PA2589,12,13,24,4,10,119,1,2,19,0,0,203,host,gene-PA2589
1,gene-PA4119,14,15,14,17,21,96,9,4,10,6,3,175,host,aph
2,gene-PA2246,30,21,8,9,21,101,1,2,8,0,2,72,host,bkdR


Ceyssens_non-directional_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,10_R1,10_R2,35_R1,35_R2,Entity,Symbol
0,gene-PA1715,3,22,0,0,0,1,host,pscB
1,gene-PA2803,7,2,0,0,0,0,host,gene-PA2803
2,gene-PA5220,7,7,0,0,0,0,host,gene-PA5220


Guegler_T7_minusToxIN_full_raw_counts_cleaned.tsv


...

,Geneid,0_R1,0_R2,0_R3,30_R1,30_R2,30_R3,45_R1,45_R2,45_R3,75_R1,75_R2,75_R3,135_R1,135_R2,135_R3,Entity,Symbol
0,gene-FTB24_03575,117,120,181,54,118,103,105,101,93,79,118,127,149,220,261,host,dpsA
1,gene-FTB24_07525,17,14,127,16,8,57,20,20,33,18,26,48,32,139,173,host,asrB
2,gene-FTB24_13070,24,13,111,17,21,46,21,26,25,22,26,40,41,145,104,host,spoIIIAE


Sprenger_VC_delta_tdh_VP882_WT_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,15_R2,15_R3,60_R1,60_R2,60_R3,120_R1,120_R2,120_R3,Entity,Symbol
0,gene-VC_RS11855,702,820,902,881,962,978,843,1112,1030,1019,host,rnc
1,gene-VC_RS08875,619,788,926,875,641,541,580,563,505,514,host,ybgC
2,gene-VC_RS06875,1202,1273,1493,1552,1202,1201,1030,1120,976,1134,host,gene-VC_RS06875


Wolfram-Schauerte_full_raw_counts_cleaned.tsv


,Geneid,0_R1,0_R2,0_R3,1_R1,1_R2,1_R3,4_R2,4_R3,7_R1,7_R2,7_R3,20_R1,20_R2,20_R3,Entity,Symbol
0,gene-b1314,88,111,35,104,73,44,28,17,5,21,9,5,8,2,host,ycjR
1,gene-b1821,1691,3221,1202,6584,5665,3613,1001,1036,349,915,444,261,450,107,host,mntP
2,gene-b4418,2455,3266,1637,3663,2095,2670,868,1038,173,541,305,113,156,42,host,sraB
